Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Is your problem regression or classification?
- [ ] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
    - Regression: Is the target right-skewed? If so, you may want to log transform the target.
- [ ] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency >= 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
    - Regression: Will you use mean absolute error, root mean squared error, R^2, or other regression metrics?
- [ ] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

If you haven't found a dataset yet, do that today. [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2) and choose your dataset.

Some students worry, ***what if my model isn't “good”?*** Then, [produce a detailed tribute to your wrongness. That is science!](https://twitter.com/nathanwpyle/status/1176860147223867393)

In [10]:
#first things first we gotta read this thing in.  It is massive, and literally split into
#three datasets.  Two of which make sense to me, one which is really weird.  So I'm just going
#to read these in.
#Reference: https://www.kaggle.com/c/bosch-production-line-performance/data
import pandas as pd

#the dots represent how many folders up we have to go before looking for the file
folder = '../../DS-Unit-2-Build/bosch-production-line-performance/'

#we are going to read in chunks of 1k for right now to keep from overloading my ram
cat_iter = pd.read_csv(folder + 'train_categorical.csv', iterator = True, chunksize = 1000)

num_iter = pd.read_csv(folder + 'train_numeric.csv', iterator = True, chunksize = 1000)

date_iter = pd.read_csv(folder + 'train_date.csv', iterator = True, chunksize = 1000)

In [11]:
#we got some chonky iterators.  Lets get those chonks!

cat_df = cat_iter.get_chunk()
num_df = num_iter.get_chunk()
date_df = date_iter.get_chunk()

/home/felix/anaconda3/envs/anaconda_env/lib/python3.8/site-packages/IPython/core/async_helpers.py:68: DtypeWarning: Columns (96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,335,407,410,413,416,419,422,425,428,431,434,437,440,443,446,449,452,455,458,461,464,467,470,473,476,479,482,485,488,491,494,497,500,503,506,509,512,515,518,611,614,617,620,623,626,629,632,635,638,641,644,647,650,653,656,659,662,665,668,671,674,677,680,683,686,689,692,695,698,701,704,707,710,713,716,719,722,1005,1007,1008,1010,1011,1013,1014,1016,1017,1019,1020,1022,1023,1025,1026,1028,1029,1031,1032,1034,1035,1037,1038,1040,1041,1043,1044,1046,1047,1049,1050,1052,1126,1129,1132,1135,1138,1141,1144,1147,1183,1185,1188,1191,1194,1197,1200,

In [12]:
#that warning is to be expected.  I can probably safely treat my numeric data set as floats
#and i'm guessing my categorical dataset should be strings, but we really need to take a look
#to find out.
print(cat_df.shape)
cat_df.head()

(1000, 2141)


,Id,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#oh definitely, lets just say all these are floats.  And heeeeeeyyyyyy there is our target!
print(num_df.shape)
num_df.head()

(1000, 970)


,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [14]:

#this one is a weirdo.  I read the reference, and it is a little strange. so L0_S0_D1 is the
#timestamp recorded at L0_S0_F0, and L0_S0_D3 is the timestamp from L0_S0_F2.  What a weirdo
#way to encode that. I assume there is some reason why.
print(date_df.shape)
date_df.head()

(1000, 1157)


,Id,L0_S0_D1,L0_S0_D3,L0_S0_D5,L0_S0_D7,L0_S0_D9,L0_S0_D11,L0_S0_D13,L0_S0_D15,L0_S0_D17,...,L3_S50_D4246,L3_S50_D4248,L3_S50_D4250,L3_S50_D4252,L3_S50_D4254,L3_S51_D4255,L3_S51_D4257,L3_S51_D4259,L3_S51_D4261,L3_S51_D4263
0,4,82.24,82.24,82.24,82.24,82.24,82.24,82.24,82.24,82.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,602.64,602.64,602.64,602.64,602.64,602.64,602.64,602.64,602.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Choose your target
My target is ```Response``` in the numerical dataset (of all places).  It is a binary categorical of 0 and 1.<br>
1 = Failed QC<br>
0 = Passed QC

# Classification
I'm ultimately looking to classify each unique ID in the test set to a pass or failed

# Target Distribution
Target is binary, so only two classes<br>
Target is pretty imbalanced, but that is to be expected for quality checks<br>

In [17]:
#Okay, I feel like I can read in the entier Response column to get an idea of how my datat is
#distributed

#okay read in only the response column and squeeze it into a series.
response = pd.read_csv(folder + 'train_numeric.csv', usecols = ['Response'], squeeze = True)
print(response.shape)
response.head()

(1183747,)


0    0
1    0
2    0
3    0
4    0
Name: Response, dtype: int64

In [18]:
#okay that looks really imbalanced, but thats not crazy for the type of problem.
#if you were failing qc in a manufacturing environment a "balanced" amount of time you would
#just be bleeding money out.
response.value_counts()

0    1176868
1       6879
Name: Response, dtype: int64

# Evaluation Metric
So accuracy isn't going to cut it for our needs.  Lets just do a confusion matrix and figure this out.

|                 | Predicted Failed                                                                                                                                          | Predicted Passed                                                                                                                                     |
|-----------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------|
| Actually Failed | Sweet                                                                                                                                                     | Part goes to production, product made, possibly fails in field, loss of consumer faith, high complexity cost and monetary cost of reverse logistics. |
| Actually Passed | Part gets binned for rework.  Best case, is identified as good during rework. Worst case classified as wastage. I'd expect lower overall cost and impact. | Sweet                                                                                                                                                |

Okay I think we figured it out.  False positive is our biggest concern here.  Not at the expense of false negatives, they do have some cost.  So I'm thinking we highly prioritize precision, but maybe lets say an 80/20 split for now.  I'll re-evaluate as time goes by.

# Choose observations to train, validate and test
#### Identifying Outliers
This is likely going to take its own entire notebook, so it might be outside of the scope of this assignment tonight. The resource I got the data from indicates there may be outliers, so I will definitely have to spend a lot of time checking.
#### What kind of split
If I do a split (I really like cross validation), I'm likely to do a random split probably 80/20.  There are date timestamps in their own dataframe, but I don't expect a time split would be useful for my purposes.  I suspect that time doesn't play a huge predictive factor, but that is a hip shot right now.